# Train 데이터에 있는 토큰을 분석하는 부분이다.
get-tokens에서 토큰화를 한 파일을 가지고 하면 된다.

In [3]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns


In [4]:
# mount to drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
# import text data from ../raw-data/ directory
os.getcwd()
data_dir = os.getcwd() + '/drive/MyDrive/aiffel/dlthon/raw-data'
print(data_dir)

/content/drive/MyDrive/aiffel/dlthon/raw-data


In [17]:
train = pd.read_csv(data_dir + '/train-tokens.csv')
train.head(5)

,idx,class,conversation,morphs,nouns
0,0,협박 대화,지금 너 스스로를 죽여달라고 애원하는 것인가? 아닙니다. 죄송합니다. 죽을 거면...,"지금,너,스스로,를,죽이,어,달라,고,애원,하,는,것,이,ㄴ가,?,아니,ㅂ니다,.,...","너,스스로,애원,거,혼자,우리,사건,선택,가족,선택권,너와,한번,겠군,이의,저,발"
1,1,협박 대화,길동경찰서입니다. 9시 40분 마트에 폭발물을 설치할거다. 네? 똑바로 들어 한번만...,"길동,경찰서,일,ㅂ니다,.,9,시,40,분,마트,에,폭발물,을,설치,하,ㄹ,거,이,...","길동,길동경찰서,경찰서,9,9시,시,40,40분,분,마트,폭발물,설치,거,한번,얘기..."
2,2,기타 괴롭힘 대화,너 되게 귀여운거 알지? 나보다 작은 남자는 첨봤어. 그만해. 니들 놀리는거 재미없...,"너,되게,귀엽,ㄴ,거,알,ㄹ지,?,나,보다,작,은,남자,는,첨,보,았,어,.,그만하...","너,거,나,남자,첨,니들,지영,키,160,재,군대,보태,보태준거,준거,난쟁이,희망,..."
3,3,갈취 대화,어이 거기 예?? 너 말이야 너. 이리 오라고 무슨 일. 너 옷 좋아보인다? 얘 돈...,"어이,거기,예,??,느,어,말,이야,느,어,.,이리,오라,이,고,무슨,일,.,느,어...","어이,거기,예,말,오라,일,옷,얘,돈,돈,너,오늘,피시,피시방,방,콜,마지막,기회,거"
4,4,갈취 대화,저기요 혹시 날이 너무 뜨겁잖아요? 저희 회사에서 이 선크림 파는데 한 번 손등에 ...,"저기,요,혹시,날,이,너무,뜨겁,잖아요,?,저희,회사,에서,이,선,크림,팔,는데,한...","저기,날,저희,회사,선,선크림,크림,번,손등,진짜,필요,중,여기,성분,거,저,피부,..."


In [19]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3950 entries, 0 to 3949
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   idx           3950 non-null   int64 
 1   class         3950 non-null   object
 2   conversation  3950 non-null   object
 3   morphs        3950 non-null   object
 4   nouns         3945 non-null   object
dtypes: int64(1), object(4)
memory usage: 154.4+ KB


## 클래스 별로 토큰을 분석해보도록 하자.
다음 두 종류로 만들어진 토큰을 살펴보자.
- 형태소 별 토큰(morphs 칼럼)
- 명사 별 토큰(nouns 칼럼)

협박 대화 = 0,
갈취 대화 = 1,
직장 내 괴롭힘 대화 = 2,
기타 괴롭힘 대화 = 3,
일반 대화 = 4

In [54]:
# 데이터셋에서 토큰을 가져오는 함수 정의
def get_tokens(dataset):
    tokens = []
    for sentence in dataset:
        for word in str(sentence).split(','):
            tokens.append(word)
    return tokens

In [55]:
# 형태소 토큰을 클래스별로 가져온다.
morph_tokens_0 = get_tokens(train[train['class'] == '협박 대화']['morphs'])
morph_tokens_1 = get_tokens(train[train['class'] == '갈취 대화']['morphs'])
morph_tokens_2 = get_tokens(train[train['class'] == '직장 내 괴롭힘 대화']['morphs'])
morph_tokens_3 = get_tokens(train[train['class'] == '기타 괴롭힘 대화']['morphs'])

print(len(morph_tokens_0))
print(len(morph_tokens_1))
print(len(morph_tokens_2))
print(len(morph_tokens_3))

128857
123908
132713
134220


In [64]:
# 명사 토큰을 클래스별로 가져온다.
noun_tokens_0 = get_tokens(train[train['class'] == '협박 대화']['nouns'])
noun_tokens_1 = get_tokens(train[train['class'] == '갈취 대화']['nouns'])
noun_tokens_2 = get_tokens(train[train['class'] == '직장 내 괴롭힘 대화']['nouns'])
noun_tokens_3 = get_tokens(train[train['class'] == '기타 괴롭힘 대화']['nouns'])

print(len(noun_tokens_0))
print(len(noun_tokens_1))
print(len(noun_tokens_2))
print(len(noun_tokens_3))

24340
25255
27239
26572


In [71]:
# 토큰의 빈도수를 분석해보자.
from collections import Counter

def token_counter(tokens, top_n:int=None):
    # top_n이 None이면 모든 토큰의 빈도수를 세서 반환한다.
    # top_n이 100이면 가장 많은 100개 토큰의 빈도수를 세서 반환한다.
    counter = Counter(tokens)
    if(top_n == None):
        return counter
    else:
        return counter.most_common(top_n)

In [72]:
# 형태소 토큰의 빈도수 보자
print("빈도수 별로 나타내는 협박 대화의 형태소 토큰")
print(token_counter(morph_tokens_0))
print()
print("빈도수 별로 나타내는 갈취 대화의 형태소 토큰")
print(token_counter(morph_tokens_1))
print()
print("빈도수 별로 나타내는 직장 내 괴롭힘 대화의 형태소 토큰")
print(token_counter(morph_tokens_2))
print()
print("빈도수 별로 나타내는 기타 괴롭힘 대화의 형태소 토큰")
print(token_counter(morph_tokens_3))
print()

빈도수 별로 나타내는 협박 대화의 형태소 토큰
Counter({'.': 5784, '어': 5734, '하': 4358, '이': 3656, '?': 3388, '아': 2467, '는': 2225, '가': 2161, '고': 1949, 'ㄹ': 1707, '야': 1677, '나': 1516, '었': 1498, 'ㄴ': 1418, '을': 1337, '지': 1312, '거': 1301, '에': 1223, '게': 1175, '니': 1105, '네': 957, '면': 948, '너': 941, '도': 907, '내': 892, '다': 880, '주': 824, '알': 823, '것': 811, '겠': 798, '있': 793, '은': 775, '말': 767, '느': 707, '죽': 699, '안': 692, '보': 691, '저': 687, '!': 671, '그러': 670, '만': 615, '그': 614, '왜': 596, '아니': 590, '없': 560, '세요': 533, '되': 531, '를': 501, '시': 499, 'ㅂ니다': 480, '죽이': 477, '버리': 457, '어요': 449, '그리하': 423, '요': 419, '지금': 400, '의': 395, '습니다': 394, '로': 391, '어서': 385, '뭐': 374, '들': 371, '냐': 353, '죄송': 351, '았': 343, '진짜': 341, '라': 341, '못': 340, '는데': 339, '내가': 337, '수': 337, '으로': 329, '여': 328, '우리': 326, '으면': 321, '싶': 318, '자': 317, '더': 310, '사람': 308, '마': 300, '같': 299, '무슨': 273, '어야': 272, '한테': 267, '좀': 267, '기': 265, '않': 265, '오': 262, '까지': 255, 'ㄴ데': 254, '그냥': 252, '니까': 2

In [73]:
# 명사 토큰의 빈도수 보자
print("빈도수 별로 나타내는 협박 대화의 명사 토큰")
print(token_counter(noun_tokens_0))
print()
print("빈도수 별로 나타내는 갈취 대화의 명사 토큰")
print(token_counter(noun_tokens_1))
print()
print("빈도수 별로 나타내는 직장 내 괴롭힘 대화의 명사 토큰")
print(token_counter(noun_tokens_2))
print()
print("빈도수 별로 나타내는 기타 괴롭힘 대화의 명사 토큰")
print(token_counter(noun_tokens_3))
print()

빈도수 별로 나타내는 협박 대화의 명사 토큰
Counter({'거': 625, '내': 521, '나': 506, '너': 484, '말': 364, '니': 325, '저': 314, '뭐': 265, '야': 261, '수': 253, '내가': 250, '우리': 230, '사람': 213, '발': 180, '마': 176, '일': 170, '안': 165, '때': 155, '하나': 146, '터': 145, '지금': 143, '어디': 139, '생각': 138, '새끼': 129, '여기': 129, '줄': 124, '칼': 123, '전': 123, '무엇': 117, '돈': 116, '집': 114, '데': 111, '이것': 111, '이': 109, '당장': 108, '미안': 106, '신고': 103, '정신': 103, '시간': 97, '가족': 95, '당신': 95, '이거': 91, '앞': 88, '경찰': 87, '오늘': 85, '한번': 83, '번': 82, '소리': 82, '애': 81, '날': 80, '라': 77, '입': 75, '하': 74, '때문': 73, '협박': 70, '주세': 68, '건': 66, '누': 66, '목숨': 65, '눈': 64, '고': 64, '식': 62, '손': 62, '이번': 60, '누구': 60, '해': 59, '대로': 58, '친구': 56, '그것': 56, '얘기': 53, '할거': 53, '장난': 52, '짓': 52, '딸': 52, '진짜': 49, '장기': 49, '부탁': 47, '놈': 47, '소리야': 47, '자식': 47, '잘못': 46, '응': 45, '말씀': 45, '제가': 44, '조금': 44, '하면': 43, '그거': 42, '세상': 42, '얼굴': 42, '엄마': 42, '넌': 42, '아이': 42, '씨': 41, '나도': 41, '거지': 40, '세요': 40, '둘': 40, '

In [ ]:
# 이제 이런 토큰들을 바탕으로 일반 문장을 만들어내야한다.